Build a Project Management Assistant using OpenAI API function calling.

In [ ]:
import openai
from openai import OpenAI

api_key = ""
api_base = "https://openai.vocareum.com/v1"

client = OpenAI(api_key=api_key, base_url=api_base)

In [2]:
import json
import pandas as pd

In [ ]:
# Load the project management data
df = pd.read_csv("project_management.csv")

In [6]:
df

,Task ID,Task Name,Project ID,Assigned To,Status,Priority,Due Date,Date Created,Last Updated,Time Estimate,Time Spent,Description,Project Phase,Dependencies
0,1,Design Database Schema,101,Jane Doe,In Progress,High,2023-08-01,2023-07-01,2023-07-10,10,4.0,Create initial database schema for customer data,Design,NaN
1,2,Implement Login Page,101,John Smith,completed,Medium,2023-08-15,2023-07-01,2023-12-15,5,NaN,"""Develop the login page UI and backend""",Implementation,1.0
2,3,Prepare Project Report,102,Alice Johnson,Completed,Low,2023-07-15,2023-06-01,2023-07-05,2,2.0,Compile the weekly project status report,Reporting,NaN


In [11]:
def task_retrieval_and_status_updates(task_id, status, last_updated):
    """Retrieve the tasks and update the task's status"""
    df.loc[df["Task ID"] == task_id, "Status"] = status
    df.loc[df["Task ID"] == task_id, "Last Updated"] = last_updated
    df.to_csv("project_management.csv", index=False)
    task = df.loc[df["Task ID"] == task_id]
    print("Task status updated")
    return json.dumps(task.to_dict())

In [12]:
def project_reporting_and_analytics(project_id):
    """Generate reports on project progress and team performance"""
    project = df.loc[df["Project ID"] == project_id]
    print("Project sent")
    return json.dumps(project.to_dict())

In [13]:
def resource_allocation_and_scheduling(
    task_id, assigned_to, time_estimate, due_date, status
):
    """Allocate tasks based on current workloads and schedules"""
    df.loc[df["Task ID"] == task_id, "Assigned To"] = assigned_to
    df.loc[df["Task ID"] == task_id, "Time Estimate"] = time_estimate
    df.loc[df["Task ID"] == task_id, "Due Date"] = due_date
    df.loc[df["Task ID"] == task_id, "Status"] = status
    df.to_csv("project_management.csv", index=False)
    print("Tasks are allocated and scheduled")
    return json.dumps(task.to_dict())

In [22]:
def run_conversation():
    # Messages is a list of initial conversation messages. The system message describes the role of the assistant. The second message is from the user, the user prompt
    messages = [
        {
            "role": "system",
            "content": "You are a project management assistant with knowledge of project statuses, task assignments, and scheduling. You can provide updates on projects, assign tasks to team members, and schedule meetings. You understand project management terminology and are capable of parsing detailed project data. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.",
        },
        {
            "role": "user",
            "content": "Change the status of task 1 to halted.",
        },  # this prompt should call task_retrieval_and_status_updates
    ]
    # Tools is a list of functions that the assistant can use. Each function is described by its name, description, and parameters.
    tools = [
        {
            "type": "function",
            "function": {
                "name": "task_retrieval_and_status_updates",
                "description": "Retrieve and update task status",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "task_id": {
                            "type": "integer",
                            "description": "The unique identifier for the task",
                        },
                        "status": {
                            "type": "string",
                            "description": "The new status of the task",
                        },
                        "last_updated": {
                            "type": "string",
                            "description": "The date of the last status update or change to the task",
                        },
                    },
                    "required": ["task_id", "status", "last_updated"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "project_reporting_and_analytics",
                "description": "Generate reports on project progress and team performance",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "project_id": {
                            "type": "integer",
                            "description": "The unique identifier for the project",
                        }
                    },
                    "required": ["project_id"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "resource_allocation_and_scheduling",
                "description": "Allocate tasks based on current workloads and schedules",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "task_id": {
                            "type": "integer",
                            "description": "The unique identifier for the task",
                        },
                        "assigned_to": {
                            "type": "string",
                            "description": "The user ID or name of the person to whom the task is assigned",
                        },
                        "time_estimate": {
                            "type": "integer",
                            "description": "An estimate of the time required to complete the task",
                        },
                        "due_date": {
                            "type": "string",
                            "description": "The deadline for the task completion",
                        },
                        "status": {
                            "type": "string",
                            "description": "The current status of the task",
                        },
                    },
                    "required": [
                        "task_id",
                        "assigned_to",
                        "time_estimate",
                        "due_date",
                        "status",
                    ],
                },
            },
        },
    ]

    # Generate response from the model
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # Let the model decide which tool (function) to use
    )

    # List the available functions and their corresponding python functions
    available_functions = {
        "task_retrieval_and_status_updates": task_retrieval_and_status_updates,
        "project_reporting_and_analytics": project_reporting_and_analytics,
        "resource_allocation_and_scheduling": resource_allocation_and_scheduling,
    }
    # Reponse message and tool_calls extract the first response message and any tool calls from response.
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    print(tool_calls)

    # End of first response, now are parse the response and call the functions the model identified from out tool list
    # Check if the model wanted to call a function
    if tool_calls:
        messages.append(
            response_message
        )  # Extend the conversation with the first response
        # Send the info for each function call and function response to the model
        for tool_call in tool_calls:  # Iterate through the tool calls in the response
            function_name = (
                tool_call.function.name
            )  # Get the name of the function that we need to call
            function_to_call = available_functions[
                function_name
            ]  # Get the function to call from the available functions mapping
            function_args = json.loads(
                tool_call.function.arguments
            )  # Convert the arguments of the function call from a JSON formatted string into a Python dictionary.
            if function_name == "task_retrieval_and_status_updates":
                function_response = function_to_call(  # call the function with the arguments. The result of the function call is stored in function_response
                    task_id=function_args.get("task_id"),
                    status=function_args.get("status"),
                    last_updated=function_args.get("last_updated"),
                )
            elif function_name == "project_reporting_and_analytics":
                function_response = function_to_call(
                    project_id=function_args.get("project_id")
                )
            elif function_name == "resource_allocation_and_scheduling":
                function_response = function_to_call(
                    task_id=function_args.get("task_id"),
                    assigned_to=function_args.get("assigned_to"),
                    time_estimate=function_args.get("time_estimate"),
                    due_date=function_args.get("due_date"),
                    status=function_args.get("status"),
                )

            message_to_append = {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,  # send the function response to the model, it's the JSON string of the function response
            }

            messages.append(message_to_append)

        messages[1]['content'] = ""  # clear the first message (parsing bug)

        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response

In [23]:
print(run_conversation())

[ChatCompletionMessageFunctionToolCall(id='call_yIW0jNhK7OmVbuF7fxkqYCKI', function=Function(arguments='{"task_id":1,"status":"halted","last_updated":"2023-07-15"}', name='task_retrieval_and_status_updates'), type='function')]
Task status updated
ChatCompletion(id='chatcmpl-CZKBu4S81PQ9mrXbEe0wFAKrAcW5e', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The task "Design Database Schema" with Task ID 1 has been updated to a status of "halted" as of July 15, 2023. The task is currently assigned to Jane Doe and is part of Project ID 101. The due date for this task is August 1, 2023, and it falls within the "Design" phase of the project. If you need to make any further changes or updates, feel free to let me know!', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1762535130, model='gpt-3.5-turbo-1106', object='chat.completion', service_tier='default', system_fingerprint='fp_98203